In [1]:
!pip install -U -q inseq sentencepiece typer datasets transformers evaluate torchinfo accelerate "neptune-client<1.0.0" nlpaug rouge_score unbabel-comet nltk

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.2 which is incompatible.
flax 0.6.7 requires rich>=11.1, but you have rich 10.16.2 which is incompatible.


# Report

Note: In this report, the plural personal pronoun was used (e.g *we* conducted experiments). The work was done individually, it is only a tribute to the practice of writing scientific articles.

## Setup  

Setup some auxiliary tools, login to hub, load training script, etc.

In [8]:
!nvidia-smi

Thu Mar 23 18:09:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Evaluation script is unable to read only UNF-8 enc.

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())

UTF-8
UTF-8


Login to huggingface_hub to be able load and download models of a user

In [ ]:
import huggingface_hub
huggingface_hub.login('TBA')

All extepiments are tracked via neptune.ai:  
https://new-ui.neptune.ai/k4black/jb-internal-representations  
So we need to login via api token:

In [ ]:
%env NEPTUNE_PROJECT=k4black/jb-internal-representations
%env NEPTUNE_API_TOKEN=

Colab was used as a free-gpu 😃 server for training and analysis tools.  
However, training was done using a separate training script. 
The following files should be copied from the repo: 
* `train.py`
* `params.json`

The training script allows to train a model (using a pretrained Hugginface Hub model) and push it to the Hub (if needed). The script requires `--config-name` parameter to select parameters from `params.json`.  
Please check the training script `--help` and source for more information on training.

In [12]:
!python train.py --help

2023-03-23 18:09:40.361460: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-23 18:09:40.361608: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-23 18:09:40.361640: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
IS_CUDA_AVAILABLE True
IS_BF16_AVAILABLE False
Usage: train.py [OPTIONS]

Options:
  --base-model TEXT               Pretrained model
                                 

## Dataset
The original dataset repo does not contain test-set labels, however, the labels can be obtained using the original scripts. So `AhmedSSoliman/CodeXGLUE-CONCODE` dataset with generated labels was used.

In [4]:
from datasets import load_dataset

In [5]:
# dataset = load_dataset("code_x_glue_tc_text_to_code")  # does not contain test labels/code 
dataset = load_dataset('AhmedSSoliman/CodeXGLUE-CONCODE')
dataset = dataset.rename_column('nl', 'text')  # just for convinience
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/AhmedSSoliman___json/AhmedSSoliman--CodeXGLUE-CONCODE-602c7546a83001c5/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['code', 'text'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['code', 'text'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['code', 'text'],
        num_rows: 2000
    })
})

## Metrics

Looking at the various benchmarks the following frequently used metrics are highlighted:

* `exact_match` - exact match of generated code. 
* `rouge` - popular translation/summarication metric - indicated n-grams overlap between generated and target text 
* `bleu` - popular translation mertic - caclulate matching n-grams   
* `pass` (valiations) - code generation metrics - depends on the number of passed unit-tests.

We don't have unittests to pass, so `exact_match`, `rouge` (without rougeSum), and `bleu` were selected to mesure during training. However later the `rouge` metric was hidden (still can be foung in [neptune.ai project](https://new-ui.neptune.ai/k4black/jb-internal-representations)) and `exact_match` and `bleu` only was considered.

The [CodeXGLUE ConCode benchmark](https://github.com/microsoft/CodeXGLUE/tree/main/Text-Code/text-to-code) uses `bleu`, `CodeBleu` and `exact_match` metrics. Banchmark provides the evaluation script which we eill be use in test set evaluation.

In [ ]:
!wget https://raw.githubusercontent.com/microsoft/CodeXGLUE/main/Text-Code/text-to-code/evaluator/bleu.py
!wget https://raw.githubusercontent.com/microsoft/CodeXGLUE/main/Text-Code/text-to-code/evaluator/evaluator.py

The script require predictions and references to be in `.txt` and `.json` format respectively (it's a bit strange, yep)

In [7]:
dataset['test'].to_json('answers.json')

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

1896530

## Train models
Train models using `train.py` script.  
This section contains mostly report, all experiments can be found in [neptune.ai project](https://new-ui.neptune.ai/k4black/jb-internal-representations).

In [ ]:
!python train.py --base-model=t5-small --config-name=default --push-to-hub

### Baseline

Only `small` versions were considered as we have limited resourses. Quality is expected to increase as we move into larger models.  
A number of experiments were conducted with the following models (check `baseline` tag in neptune.ai for exact numbers):

| model | description | baseline |
| --- | --- | --- | 
| `t5-small` | classical `t5` model, english-language pretrained | low quality, but can be applied | 
| `t5-v1.1-small` | updated `t5` model, shoul be a bit better | unable to use with `fp16`, so can not fit into the gpu memory | 
| `flan-t5-small` | propmts-pretrainined version of the `t5` model, trained using some amount of code data | quality is lower than `t5`, `fp16` not working, but can be applied |
| `google/byt5-small` | byte-level `t5` model, trained using some amount of code data | due to increased number of tokens can not fit into the gpu memory |
| `Salesforce/codet5-small` | `t5` model trained on code tasks similarly to the original `t5` | best baseline model so far, much better then pure `t5` | 
| `epinnock/flan-t5-small-codeparrot-xlcost-text-to-code` | `flan-t5` model finetuned for the `xlcost-text-to-code` dataset | low quality |

For far the `Salesforce/codet5-small` was choosen for the further experiments.




### Parameters and Tricks 

We searched for some T5 training advice in [HugginFace discussions](https://discuss.huggingface.co/t/t5-finetuning-tips/684). The following tricks were considered:
* higher learning rate of `1e-3`/`1e-4` (optimal was `5e-4` for `t5` and `1e-4` for `codet5`)
* t5 models are not working with `fp16` (actually, `t5` and `codet5` work with it, others - don't)
* `AdaFactor` optimizer is better for T5 model (we could not find a significant difference with `AdamW`, it's even a bit better, so it was chosen) 

From our experience in training LM, we used some other tricks:
* `fp16` is a savior, horray training 2-times decay! (if applicable)
* `warmup_ratio=0.05` - to warm up a model
* `early_stopping` - with 5 steps patience (however metrics are not much reliable, so validation loss was used) 
* `torch_compile=False`/`bf16=False` - as the colab GPUs don't support it =(
* all random seeds are fixed with `42` (it was Deep Thought)

Some other ideas were used:
* Dataset contains special tokens `concode_elem_sep`/`concode_field_sep`, which we added as new special tokens (and new embeddings to be trained), as it shows the small quality increase and reduce input length
* `generation_num_beams=1` for validation as it's not much important for validation and simply speed up evaluation 
* `generation_max_length=20` **for validation (due to error)**. By mistake, such a parameter was left with the default value, which lowers and limits the validation metrics, but thinly leaves them representative. Due to the fact that a number of experiments were conducted with =20, such a parameter was left in place to be able to compare the models, when `256` was used for the final **test evaluation**.

### Finetuning

After selecting the `Salesforce/codet5-small` model, we tested a number of experiments in the learning rate:

| learning rate | Exact Match | BLUE |
| --- | --- | --- |
| 1e-3 | 0.156 | 0.1246 |
| 5e-4 | 0.146 | 0.1264 |
| 3e-4 | 0.156 | 0.1297 |
| 2e-4 | 0.158 | 0.1361 |
| 1e-4 | **0.163** | **0.1335** |
| 5e-5 | 0.163 | 0.1345 |

Bold font model was selected for further experiments. 

Note: Not all of the models' converged at the time of the training stop, but the general tendency is similar.

Regarding special tokens (`concode_elem_sep`/`concode_field_sep`) usage:

| add special tokens | Exact Match | BLUE |
| --- | --- | --- |
| without | 0.163 | 0.1335 |
| with | **0.166** | **0.1368** |

---

So the model with lerning rate of `1e-4` and using special tokens was selected as best one.
The model available via HugginfaceHub as `k4black/Salesforce-codet5-small-CodeXGLUE-CONCODE-w_special_tokens`



## Predict
Predict with selected model and evaluate using the [official script](https://github.com/microsoft/CodeXGLUE/tree/main/Text-Code/text-to-code).


In [8]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

In [9]:
model_name = 'k4black/Salesforce-codet5-small-CodeXGLUE-CONCODE-w_special_tokens'
model_pipeline = pipeline("text2text-generation", model_name, device=0)

In [10]:
predictions = []
for out in tqdm(
    model_pipeline(
        KeyDataset(dataset['test'], 'text'), 
        batch_size=8, 
        truncation='only_first', 
        clean_up_tokenization_spaces=False,
        max_length=256,
        num_beams=8,
        ), 
    total=len(dataset['test'])
    ):
    predictions.append(out[0]['generated_text'])

with open('predictions.txt', 'w', encoding='utf-8') as f:
    for p in predictions:
        f.write(p)
        f.write('\n')

  0%|          | 0/2000 [00:00<?, ?it/s]

### Results
Evaluate obtained code with official banchmark script.

We did not use beam-seach during training, as it slows down the inference, but should improve quality regardless of the model (to some extent). So for the final metric we tried a couple of variants of `num_beams` an choose `4 beams` prediction (2 beams will work fine as well).

In [83]:
!python evaluator.py -a=answers.json -p=predictions.txt
# k4black/Salesforce-codet5-small-CodeXGLUE-CONCODE-w_special_tokens
# max_length = 256 num_beams = 1

INFO:__main__:BLEU: 31.57, EM: 21.15


In [79]:
!python evaluator.py -a=answers.json -p=predictions.txt
# k4black/Salesforce-codet5-small-CodeXGLUE-CONCODE-w_special_tokens
# max_length = 256 num_beams = 2

INFO:__main__:BLEU: 33.81, EM: 21.3


In [87]:
!python evaluator.py -a=answers.json -p=predictions.txt
# k4black/Salesforce-codet5-small-CodeXGLUE-CONCODE-w_special_tokens
# max_length = 256 num_beams = 4

INFO:__main__:BLEU: 34.96, EM: 21.4


In [15]:
!python evaluator.py -a=answers.json -p=predictions.txt
# k4black/Salesforce-codet5-small-CodeXGLUE-CONCODE-w_special_tokens
# max_length = 256 num_beams = 8

INFO:__main__:BLEU: 34.86, EM: 21.35


---

The scores seems to be pretty hight, as 
* [official dataset page](https://github.com/microsoft/CodeXGLUE/tree/main/Text-Code/text-to-code) provides authors `CodeGPT-adapted` model at `BLEU: 32.79, EM 20.10`
* [official banchmark page](https://microsoft.github.io/CodeXGLUE/) shows highest result with `StructCoder` model at `BLEU: 40.91, EM: 22.35`

So scores `BLEU: 34.96, EM: 21.4` (n_beams=4) obtained with `small`-sized model are promising and should scale as model size grow.  
The validation charts do not show overfitting, so it seems even a small-sized model can bit tuned a bit higher with some text augumentation (input description only) and more experiments on learning rate and special tokens.

### Analysis 
Perform error analysis using "the stare method" and trying to use [inseq](https://github.com/inseq-team/inseq) library.

Firstly, lets see what we will work with by just printing all wrong predictions (as we have only 20% exactly matched, it'll be a lot)

In [12]:
import random

In [13]:
for i, (p, t, c) in enumerate(zip(predictions, dataset['test']['text'], dataset['test']['code'])):
    if c != p:
        print(i)
        print('TEXT:', len(model_pipeline.tokenizer.encode(t)), '\t', t)
        print('PRED:', len(model_pipeline.tokenizer.encode(p)), '\t', p)
        print('CODE:', len(model_pipeline.tokenizer.encode(c)), '\t', c)
        print()

0
TEXT: 102 	 generate mappings for each function node and parameters and variables names associated with it . concode_field_sep int parentScope concode_elem_sep ArrayList functionBracePositions concode_elem_sep ObjArray funcObjects concode_elem_sep int functionNum concode_elem_sep ArrayList functionVarMappings concode_elem_sep int lastTokenCount concode_elem_sep ArrayList replacedTokens concode_field_sep boolean isInScopeChain concode_elem_sep void reset concode_elem_sep void leaveNestingLevel concode_elem_sep String getMappedToken concode_elem_sep String getPreviousTokenMapping concode_elem_sep void collectFuncNodes concode_elem_sep int sourceCompress concode_elem_sep void enterNestingLevel
PRED: 50 	 void function ( ) { for ( int loc0 = 0 ; loc0 < functionVarMappings . size ( ) ; loc0 ++ ) { functionVarMappings . add ( functionVarMappings . get ( loc0 ) ) ; } }
CODE: 34 	 void function ( ScriptOrFnNode arg0 ) { int loc0 = - 1 ; collectFuncNodes ( arg0 , loc0 , null ) ; }

1
TEXT: 88

Token indices sequence length is longer than the specified maximum sequence length for this model (779 > 512). Running this sequence through the model will result in indexing errors


Выходные данные были обрезаны до нескольких последних строк (5000).
733
TEXT: 167 	 returns if result is ko concode_field_sep int errorLevelFolder concode_elem_sep String tooltip concode_elem_sep int errorLevel concode_elem_sep String value concode_field_sep void restoreFolderErrorLevel concode_elem_sep void setKO concode_elem_sep void setKO concode_elem_sep boolean isWarning concode_elem_sep void setErrorLevel concode_elem_sep int getErrorLevel concode_elem_sep String analyseTrack concode_elem_sep String analyseTrackYear concode_elem_sep String formatNumber concode_elem_sep String colorField concode_elem_sep String colorField concode_elem_sep String getValue concode_elem_sep String analyseTrackComment concode_elem_sep String analyseTrackBpm concode_elem_sep int hashCode concode_elem_sep void setValue concode_elem_sep boolean equals concode_elem_sep String getDisplayToolTip concode_elem_sep void setOK concode_elem_sep String toString concode_elem_sep boolean isNotValid concode_elem_sep

Ok, eyes darting around. Let's select some random samples to analyse it

In [18]:
random.seed(42)
selected_indxes = random.sample(range(len(predictions)), k=8)
for i in selected_indxes:
    p, t, c = predictions[i], dataset['test'][i]['text'], dataset['test'][i]['code']
    print(i)
    print('TEXT:', len(model_pipeline.tokenizer.encode(t)), '\t', t)
    print('PRED:', len(model_pipeline.tokenizer.encode(p)), '\t', p)
    print('CODE:', len(model_pipeline.tokenizer.encode(c)), '\t', c)
    print()

1309
TEXT: 23 	 always returns the configured issuer url concode_field_sep String issuer concode_field_sep void afterPropertiesSet concode_elem_sep void setIssuer
PRED: 11 	 String function ( ) { return issuer ; }
CODE: 27 	 IssuerServiceResponse function ( HttpServletRequest arg0 ) { return new IssuerServiceResponse ( getIssuer ( ) , null , null ) ; }

228
TEXT: 81 	 compares to a specific object . concode_field_sep Set<InodeFile> mFiles concode_elem_sep long sTtlIntervalMs concode_elem_sep long mTtlIntervalStartTimeMs concode_field_sep void removeFile concode_elem_sep long getTtlIntervalMs concode_elem_sep int hashCode concode_elem_sep int compareTo concode_elem_sep Set<InodeFile> getFiles concode_elem_sep long getTtlIntervalStartTimeMs concode_elem_sep long getTtlIntervalEndTimeMs concode_elem_sep void addFile
PRED: 67 	 boolean function ( Object arg0 ) { if ( arg0 == this ) { return true ; } if ( ! ( arg0 instanceof InodeFile ) ) { return false ; } InodeFile loc0 = ( InodeFile ) ar

Some observations:  
* The model generally "captured" the common structure, correctly maintains bracket balance, function arguments, separates strings, etc;
* Sometimes errors is not errors, but another interpretation (e.g. #51, where `View arg0` was used as argumens, not `int arg0`) or a perfectly correct construction, but written differently (e.g. #228 `if ( arg0 == this )` instead of `if ( this == arg0 )`);
* On the other hand, the model sometimes provides a more correct option than the real answer. e.g. #1309 where the model logically returned the string `String issuer`, "assuming" that it is `configured issuer url`, rather than assuming something like `HttpServletRequest arg0`/`new IssuerServiceResponse`, which is not contained in the context;

In general, we can say that there is no particular "trend" in errors. Such a model, we believe, performed quite well for such complex task: 
* Perhaps we should revise the context a bit and add missing functions and classes from outer scopes;
* Increasing the size of the model could also help;
* For some classes it is difficult to assume their essence and application, perhaps it is necessary to add their docsrting to the context somehow (or simplified functions/classes body);

---

For interest, one might consider why a model makes certain token choices.

In [19]:
import inseq
import warnings
warnings.filterwarnings('ignore')

`inseq` allow 2 analysis types: Unconstrained Attributing (research attention scores between input and output) and Constrained (research difference between 2 fixed outputs, but require same number of tokens in samples), we will check the first one.

In [20]:
model = inseq.load_model(model_name, 'input_x_gradient')

In [22]:
id_ = 1309
pred = predictions[id_]
code = dataset['test'][id_]['code']
text = dataset['test'][id_]['text']

out = model.attribute(
    text,
    pred,
    attribute_target=True,
    step_scores=['probability']
)
print()
print('TEXT:', text)
print('PRED:', pred)
print('CODE:', code)
out.show() 

Attributing with input_x_gradient...: 100%|██████████| 12/12 [00:00<00:00, 11.37it/s]


TEXT: always returns the configured issuer url concode_field_sep String issuer concode_field_sep void afterPropertiesSet concode_elem_sep void setIssuer
PRED: String function ( ) { return issuer ; }
CODE: IssuerServiceResponse function ( HttpServletRequest arg0 ) { return new IssuerServiceResponse ( getIssuer ( ) , null , null ) ; }


,<s>,String,Ġfunction,Ġ(,Ġ),Ġ{,Ġreturn,Ġissuer,Ġ;,Ġ},</s>
<s>,0.045,0.024,0.019,0.015,0.021,0.021,0.023,0.015,0.02,0.019,0.013
always,0.076,0.076,0.1,0.07,0.112,0.112,0.158,0.062,0.07,0.089,0.067
Ġreturns,0.041,0.046,0.048,0.031,0.045,0.044,0.059,0.024,0.033,0.036,0.026
Ġthe,0.031,0.029,0.029,0.022,0.025,0.019,0.023,0.027,0.019,0.017,0.011
Ġconfigured,0.06,0.06,0.059,0.047,0.054,0.05,0.055,0.043,0.045,0.038,0.027
Ġissuer,0.082,0.087,0.067,0.069,0.069,0.053,0.058,0.127,0.097,0.045,0.02
Ġurl,0.041,0.133,0.102,0.046,0.055,0.047,0.048,0.048,0.072,0.036,0.019
Ġ,0.018,0.023,0.019,0.011,0.017,0.013,0.014,0.023,0.019,0.009,0.007
concode_field_sep,0.065,0.027,0.024,0.021,0.023,0.025,0.022,0.021,0.025,0.02,0.013
ĠString,0.035,0.05,0.031,0.023,0.028,0.026,0.023,0.019,0.038,0.015,0.011


As expected above, the model "paid attention" to the `url` token when generating the `String` return value, as well as the high gradient shown by the `String issuer` in context.

---

In [23]:
id_ = 51
pred = predictions[id_]
code = dataset['test'][id_]['code']
text = dataset['test'][id_]['text']

out = model.attribute(
    text,
    pred,
    attribute_target=True,
    step_scores=['probability']
)
print()
print('TEXT:', text)
print('PRED:', pred)
print('CODE:', code)
out.show() 

Attributing with input_x_gradient...: 100%|██████████| 25/25 [00:03<00:00,  7.44it/s]



TEXT: set an action view for this menu item . an action view will be displayed in place of an automatically generated menu item element in the ui when this item is shown as an action within a parent . concode_field_sep android.view.MenuItem mMenuItem concode_field_sep boolean hasSubMenu concode_elem_sep CharSequence getTitleCondensed concode_elem_sep int getGroupId concode_elem_sep int getItemId concode_elem_sep View getActionView concode_elem_sep View getActionView concode_elem_sep CharSequence getTitle concode_elem_sep MenuItem setChecked concode_elem_sep char getAlphabeticShortcut concode_elem_sep MenuItem setOnMenuItemClickListener concode_elem_sep MenuItem setOnMenuItemClickListener concode_elem_sep Intent getIntent concode_elem_sep boolean isCheckable concode_elem_sep MenuItem setShortcut concode_elem_sep MenuItem setVisible concode_elem_sep void setShowAsAction concode_elem_sep void setShowAsAction concode_elem_sep SubMenu getSubMenu concode_elem_sep MenuItem setIntent concode_

,<s>,MenuItem,Ġfunction,Ġ(,ĠView,Ġarg,0,Ġ),Ġ{,Ġm,MenuItem,Ġ.,ĠsetAction,View,Ġ(,Ġarg,0,Ġ),Ġ;,Ġreturn,Ġthis,Ġ;,Ġ},</s>
<s>,0.008,0.009,0.008,0.007,0.009,0.009,0.008,0.01,0.007,0.008,0.005,0.007,0.008,0.008,0.006,0.006,0.005,0.006,0.005,0.007,0.005,0.005,0.004,0.005
set,0.011,0.024,0.016,0.013,0.019,0.014,0.012,0.027,0.018,0.018,0.004,0.013,0.016,0.015,0.013,0.011,0.011,0.012,0.012,0.012,0.013,0.007,0.007,0.007
Ġan,0.009,0.008,0.006,0.016,0.013,0.012,0.01,0.017,0.011,0.012,0.004,0.009,0.011,0.008,0.009,0.009,0.01,0.01,0.008,0.009,0.005,0.004,0.004,0.006
Ġaction,0.011,0.009,0.008,0.013,0.017,0.018,0.018,0.022,0.012,0.011,0.007,0.015,0.039,0.02,0.017,0.014,0.013,0.012,0.01,0.012,0.009,0.005,0.005,0.005
Ġview,0.009,0.014,0.016,0.016,0.029,0.037,0.03,0.037,0.02,0.017,0.007,0.019,0.021,0.029,0.021,0.019,0.023,0.017,0.014,0.016,0.01,0.007,0.007,0.008
Ġfor,0.007,0.007,0.006,0.009,0.011,0.012,0.01,0.016,0.01,0.011,0.003,0.009,0.008,0.011,0.017,0.007,0.008,0.008,0.007,0.008,0.005,0.004,0.004,0.004
Ġthis,0.007,0.003,0.004,0.007,0.007,0.006,0.005,0.01,0.006,0.006,0.003,0.006,0.005,0.005,0.009,0.004,0.004,0.005,0.003,0.005,0.003,0.005,0.003,0.003
Ġmenu,0.008,0.004,0.005,0.006,0.006,0.006,0.005,0.008,0.006,0.008,0.009,0.007,0.005,0.005,0.005,0.005,0.004,0.005,0.004,0.006,0.006,0.005,0.004,0.003
Ġitem,0.005,0.004,0.004,0.004,0.004,0.005,0.004,0.006,0.004,0.004,0.003,0.005,0.003,0.003,0.004,0.004,0.003,0.003,0.003,0.004,0.003,0.003,0.002,0.002
Ġ.,0.004,0.004,0.002,0.005,0.004,0.005,0.004,0.006,0.004,0.004,0.002,0.004,0.004,0.004,0.004,0.004,0.003,0.004,0.003,0.003,0.002,0.002,0.002,0.002


Because of the rather large context, the probabilities do not stand out as clearly as in the example above. However, one can see the high attention span between the `View` arguments and the `View` class from the context. It is also interesting how the model has generalized and "invented" the `setAction` function - you can see a high attention span on `getAction` from the context.

---

In general, we can conclude from this superficial analysis that the model "pays attention" to sufficiently correct tokens in the input sequence and it has not retrained. Overall, the problem is quite complex, especially having only class names and types as context.